# Estimating FLOPs requirementsc DeepTS for neural networks.
**WORK IN PROGRESS**

In [2]:
import contextlib
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, RNN, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, Add, Flatten, SimpleRNNCell
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, TimeDistributed, LSTMCell

from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot

In [3]:
print("TensorFlow version: {}".format(tf.__version__))

TensorFlow version: 2.0.0-alpha0


In [4]:
# It's still here because DeepTS work in progress.
class ModelSummary(object):
    """
    FLOPs are computed for batch size = 1. Only Dense and Conv2D layers are taken into account.
    FLOPs are forward FLOPs (inference) and should generally be used to compare models and not
        estimating times.
    Supported layers/wrappers:
        Dense
        Conv2D
        Conv2DTranspose
        RNNs/bidirectional RNNs with the following cells: SimpleRNNCell, LSTMCell and GRUCell
        TimeDistributed with Dense, Conv2D and Conv2DTranspose

    What if batch size > 1 and need backward/training FLOPs?
       - For batch size N, multiple result by N.
       - For backward FLOPs, multiply results by 2.
       - For training FLOPs, multiply result by 3.
    """

    def __init__(self, model, verbose=False):
        """
            TODO: What if user reuses layers with functional API?
        """
        self.name = model.name
        self.layers = []  # Per-layer statistics
        self.gflops = 0.0  # Total model gFLOPs
        self.nparams = model.count_params()  # Total model parameters
        self.params_mb = 0  # Total size in MB for model parameters
        self.verbose = verbose  # If true, print layers used in computations

        for layer in model.layers:
            repeat_count = 1
            if isinstance(layer, TimeDistributed):
                repeat_count = layer.input_shape[1]  # Batch, Length, FeatureDim1, FeatureDim2, ...
                layer = layer.layer
            if isinstance(layer, Dense):
                self.compute_dense_layer_stats(layer, repeat_count)
            elif isinstance(layer, Conv2D):
                self.compute_conv2d_layer_stats(layer, repeat_count)
            elif isinstance(layer, Conv2DTranspose):
                self.compute_conv2dtranspose_layer_stats(layer, repeat_count)
            elif isinstance(layer, (RNN, Bidirectional)):
                self.compute_rnn_layer_stats(layer)

        for layer in self.layers:
            layer['gflops'] /= 1e9
            layer['params_mb'] = layer['nparams'] * 4 / (1024 * 1024)
            self.gflops += layer['gflops']
            self.params_mb += layer['params_mb']

    def add_layer(self, **kwargs):
        self.layers.append(kwargs)

    def compute_rnn_layer_stats(self, layer):
        num_steps = layer.input_shape[1]      # Number of time steps (sequence length)
        input_size = layer.input_shape[2]     # Number of input features
        output_size = layer.output_shape[-1]  # Number of output features
        layer_params = layer.count_params()   # Number of layer parameters
        layer_name = layer.name               # Name of a layer, will be qualified with cell type

        repeat_count = 1                      # If bidirectional, this will be 2
        if isinstance(layer, Bidirectional):
            if layer.merge_mode == 'concat':
                output_size = output_size // 2
            elif layer.merge_mode is None:
                raise NotImplementedError("Implement this!")
            repeat_count = 2
            layer = layer.layer

        # By default, number of later FLOPs is equal to RNN FLOPs
        layer_flops = repeat_count * (num_steps * (input_size * output_size + output_size * output_size))
        rnn_cell = None
        if isinstance(layer.cell, SimpleRNNCell):
            rnn_cell = 'SimpleRNN'
        elif isinstance(layer.cell, LSTMCell):
            rnn_cell = 'LSTM'
            layer_flops *= 4
        elif isinstance(layer.cell, GRUCell):
            rnn_cell = 'GRU'
            layer_flops *= 3

        if rnn_cell:
            self.add_layer(
                name="{} ({})".format(layer_name, rnn_cell),
                gflops=layer_flops,
                nparams=layer_params)
        if self.verbose:
            print("Found supported layer: type={}, num_steps={}, input_size={}, "
                  "output_size={}.".format(rnn_cell, num_steps, input_size, output_size))

    def compute_dense_layer_stats(self, layer, repeat_count=1):
        # Ignoring biases
        if self.verbose:
            print("Found supported layer: type=Dense, repeat_count={}.".format(repeat_count))
        self.add_layer(
            name=layer.name,
            gflops=repeat_count * (np.prod(layer.weights[0].shape)),
            nparams=layer.count_params())

    def compute_conv2d_layer_stats(self, layer, repeat_count=1):
        """
            layer.weights[0].shape  :  Filter Shape [FilterDim, FilterDim, InChannels, OutChannels]
            layer.output_shape      :  [Batch, SpatialDim, SpatialDim, OutChannels]
        """
        if self.verbose:
            print("Found supported layer: type=Conv2D, repeat_count={}.".format(repeat_count))
        # Number of flops per one output feature
        filter_flops = np.prod(layer.weights[0].shape)
        self.add_layer(
            name=layer.name,
            gflops=repeat_count * (filter_flops * layer.output_shape[1] * layer.output_shape[2]),
            nparams=layer.count_params())

    def compute_conv2dtranspose_layer_stats(self, layer, repeat_count=1):    
        """
            Double check this implementation. Consider this layer as Conv2D reversing forward/backward
            passes.
        """
        if self.verbose:
            print("Found supported layer: type=Conv2DTranspose, repeat_count={}.".format(repeat_count))
        # Number of flops per one output feature for 'depth' column
        filter_flops = np.prod(layer.weights[0].shape)
        self.add_layer(
            name=layer.name,
            gflops=repeat_count * (filter_flops * layer.input_shape[1] * layer.input_shape[2]),
            nparams=layer.count_params())

    def __str__(self):
        return "batch_size=1, forward_gflops={:.4f}, nparams={:,}".format(self.gflops, self.nparams)

    def summary(self):
        df = pd.DataFrame(self.layers + [{'name': 'TOTAL', 'gflops': self.gflops, 'nparams': self.nparams,
                                          'params_mb': self.params_mb}],
                          columns=['name', 'gflops', 'nparams', 'params_mb'])
        print(self.name)
        print(df)

In [5]:
class Model(object):
    """ A base class for all NN models. """
    def __init__(self, name):
        self.name = name

    def create(self):
        """ This method must return an instance of `tensorflow.python.keras.models.Model`. """
        raise NotImplementedError('Implement me in a derived class.')

    @staticmethod
    def Dense(size, activation='relu', **kwargs):
        return tf.keras.layers.Dense(size, activation=activation, **kwargs)

    @staticmethod
    def Input(shape, name='input'):
        return tf.keras.layers.Input(shape, name=name)

# LSTM Anomaly Detect
Example code for neural-network-based anomaly detection of time-series data (uses LSTM). Super simple and not particularly usefull ([repo](https://github.com/aurotripathy/lstm-anomaly-detect)).  
1. Sequence length = 100
2. Number of features = 1
3. Batch size = 50
4. Data is synthetic.

Three LSTM layers (64, 256, 100). Input length is 100.

In [14]:
class Model01(Model):
    """ 
    Super simple and not particularly usefull:
        https://github.com/aurotripathy/lstm-anomaly-detect
    """
    def __init__(self):
        super().__init__('Model01: LSTM anomaly detection')

    def create(self):
        return tf.keras.models.Sequential([
            Model.Input((100,1)),
            LSTM(64, return_sequences=True, name='lstm1'),
            Dropout(0.2),
            LSTM(256, return_sequences=True, name='lstm2'),
            Dropout(0.2),
            LSTM(100, return_sequences=False, name='lstm3'),
            Dropout(0.2),
            Dense(1, activation='linear', name='output')
        ], name=self.name)

In [16]:
with contextlib.redirect_stderr(None):
    ModelSummary(Model01().create()).summary()

Model01: LSTM anomaly detection
           name        gflops  nparams  params_mb
0  lstm1 (LSTM)  1.664000e-03    16896   0.064453
1  lstm2 (LSTM)  3.276800e-02   328704   1.253906
2  lstm3 (LSTM)  1.424000e-02   142800   0.544739
3        output  1.000000e-07      101   0.000385
4         TOTAL  4.867210e-02   488501   1.863483


# Keras Anomaly Detection
A repository with various models (FCNN, LSTM and Conv) for detecting anomalies ([repo](https://github.com/chen0040/keras-anomaly-detection)).
 LSTM models are defined [here](https://github.com/chen0040/keras-anomaly-detection/blob/master/keras_anomaly_detection/library/recurrent.py)

In [19]:
class Model02(Model):
    """ 
    https://github.com/chen0040/keras-anomaly-detection/blob/master/keras_anomaly_detection/library/recurrent.py#L7
    """
    def __init__(self):
        super().__init__('Model02: LSTM anomaly detection')

    def create(self):
        # Sequence length = 210, number of features = 1, batch size = ?
        return tf.keras.models.Sequential([
            Model.Input((210,1)),
            LSTM(128, return_sequences=False, name='lstm1'),
            Dense(128, activation='linear', name='output')
        ], name=self.name)

In [20]:
with contextlib.redirect_stderr(None):
    ModelSummary(Model02().create()).summary()

Model02: LSTM anomaly detection
           name    gflops  nparams  params_mb
0  lstm1 (LSTM)  0.013870    66560   0.253906
1        output  0.000016    16512   0.062988
2         TOTAL  0.013886    83072   0.316895


In [25]:
class Model03(Model):
    """ 
    https://github.com/chen0040/keras-anomaly-detection/blob/master/keras_anomaly_detection/library/recurrent.py#L218
    """
    def __init__(self):
        super().__init__('Model03: bidirectional LSTM anomaly detection')

    def create(self):
        # Sequence length = 210, number of features = 1, batch size = ?
        return tf.keras.models.Sequential([e
            Input(shape=(210,1)),
            Bidirectional(LSTM(128, return_sequences=False), name='bLSTM'),
            Dense(128, activation='linear', name='output')
        ], name=self.name)

In [26]:
with contextlib.redirect_stderr(None):
    ModelSummary(Model03().create()).summary()

Model03: bidirectional LSTM anomaly detection
           name    gflops  nparams  params_mb
0  bLSTM (LSTM)  0.027740   133120   0.507812
1        output  0.000033    32896   0.125488
2         TOTAL  0.027773   166016   0.633301


# [MTSAnomalyDetection](https://github.com/jsonbruce/MTSAnomalyDetection)
 
Multidimensional Time Series Anomaly Detection (MTSAD). Model is implemented [here](https://github.com/jsonbruce/MTSAnomalyDetection/blob/master/ensemblation/model.py).

In [31]:
class Model04(Model):
    """ 
    https://github.com/chen0040/keras-anomaly-detection/blob/master/keras_anomaly_detection/library/recurrent.py#L218
    """
    def __init__(self):
        super().__init__('Model03: bidirectional LSTM anomaly detection')

    def create(self):
        # Sequence length = 210, number of features = 1, batch size = ?
        return tf.keras.models.Sequential([
            Model.Input((210,1)),
            Bidirectional(LSTM(128, return_sequences=False), name='bLSTM'),
            Dense(128, activation='linear', name='output')
        ], name=self.name)

In [32]:
with contextlib.redirect_stderr(None):
    ModelSummary(Model04().create()).summary()

Model03: bidirectional LSTM anomaly detection
           name    gflops  nparams  params_mb
0  bLSTM (LSTM)  0.027740   133120   0.507812
1        output  0.000033    32896   0.125488
2         TOTAL  0.027773   166016   0.633301
